In [278]:
#!/usr/bin/env python
# coding: utf-8

In[1]:

In [279]:
import pandas as pd
import linopy

In [280]:
def prepare_input_data(bus, year):
    
    availability_factors = pd.concat([pd.read_hdf("/trinity/home/fuhrand/ERAA/open_eraa/resources/res_profile.h5", carrier) for carrier in res_carriers], keys=res_carriers)
    res_caps = all_caps.loc[year, res_carriers,bus, :].reset_index([0,2], drop=True)
    
    country_plants = plants[(plants.entry<=year)&(plants.exit>year)&(plants.bus==bus)]
    total_res_generation = availability_factors.loc[:, :, :, str(year), bus].unstack(0).multiply(res_caps).sum(axis=1)
    
    country_plants.index.name = "plant"
        
    max_demand = demand.loc[:, :, bus, str(year)].groupby(level=0).max()
    min_res = total_res_generation.groupby(level=0).min()
    max_residual_demand_per_week = (max_demand - min_res).groupby(max_demand.index//(24*7)).max()
    max_residual_demand_per_week.index = max_residual_demand_per_week.index*7
    max_residual_demand_per_week = max_residual_demand_per_week.reindex(day).ffill()
    
    outage_days_country_plants = number_of_days.reindex(pd.MultiIndex.from_arrays([country_plants.carrier, country_plants.age_class]))
    outage_days_country_plants.index = country_plants.index
    share_winter_country_plants = share_winter.reindex(pd.MultiIndex.from_arrays([country_plants.carrier, country_plants.age_class]))
    share_winter_country_plants.index = country_plants.index
    return country_plants, max_residual_demand_per_week, outage_days_country_plants, share_winter_country_plants, 

In [281]:
def optimize_maintenance_scheduling(max_residual_demand_per_week):
    country_plants_p_nom = country_plants.p_nom.div(1e3) 
    max_residual_demand_per_week_normed = max_residual_demand_per_week.div(1e3)
    
    m = linopy.Model()
    plant = country_plants.index
    
    maintenance = m.add_variables(binary=True, coords=[day, plant], name="maintenance")
    
    daily_margin = m.add_variables(coords=[day], name="daily_margin")
    
    days_winter = share_winter_country_plants.multiply(outage_days_country_plants).sort_values().round()
    
    m.add_constraints(
        maintenance.loc[winter_time_days].sum("day")>=days_winter,
        name="winter_maintenance_days"
    )
    
    m.add_constraints(
        maintenance.sum("day") == outage_days_country_plants,
        name="maintenance_target"
    )
    m.add_constraints(
        maintenance.sum("plant").loc[364] == 0,
        name="no_maintenance_silvester"
    )
    
    m.add_constraints(
        #daily_margin == max_residual_demand_per_week - ((1 - maintenance)*country_plants.p_nom).sum("plant"),
        daily_margin == ((1 - maintenance)*country_plants_p_nom).sum("plant") - max_residual_demand_per_week_normed + constant,
        name="daily_margin"
    )
    
    m.add_objective(daily_margin**2)
    m.solve(solver_name=solver_name, **solver_options)
    return m

In [282]:
#technology_parameters = pd.read_hdf(snakemake.input.technology_parameters)
technology_parameters = pd.read_hdf("/trinity/home/fuhrand/ERAA/open_eraa/resources/technology_parameters.h5")

In [283]:
#plants = pd.read_hdf(snakemake.input.power_plants, "detailed")
plants = pd.read_hdf("/trinity/home/fuhrand/ERAA/open_eraa/resources/capacity_tables/individual_plants.h5", "detailed")
#demand = pd.read_hdf(snakemake.input.demand)
demand = pd.read_hdf("/trinity/home/fuhrand/ERAA/open_eraa/resources/demand.h5")

In [284]:
#solver_name = snakemake.config["solving"]["solver_maintenance"]["name"]
#options = snakemake.config["solving"]["solver_maintenance"]["options"]
#solver_options = snakemake.config["solving"]["solver_options"][options]
import yaml
with open('/trinity/home/fuhrand/ERAA/open_eraa/config/config.yaml', 'r') as file:
    config_local = yaml.safe_load(file)

solver_name = config_local["solving"]["solver_maintenance"]["name"]
options = config_local["solving"]["solver_maintenance"]["options"]
solver_options = config_local["solving"]["solver_options"][options]

In [285]:
res_carriers = [ "onwind",  "offwind", "solar-track", "solar-fix", "solar-rsd",  "solar-ind",  "CSP", "CSP-stor"]

In [286]:
#save_hdf = snakemake.output.maintenance_profiles
#all_caps = pd.read_hdf(snakemake.input.all_caps)
all_caps = pd.read_hdf("/trinity/home/fuhrand/ERAA/open_eraa/resources/all_capacities.h5")

In [341]:
winter_time_days = list(range(59)) + list(range(365-31, 365))
constant = 180
day = pd.Index(range(365), name="day")
target_years = all_caps.index.levels[0]

In [288]:
number_of_days = technology_parameters.maintenance_n_days.copy()
share_winter = technology_parameters.maintenance_share_winter.copy()

In [365]:
maintenance_profiles = []
status = []
key=[]
for bus in ['UK00']: #plants.bus.unique():
    for year in target_years:
        print(bus, year)
        country_plants, max_residual_demand_per_week, outage_days_country_plants, share_winter_country_plants = prepare_input_data(bus, year)
        m =  optimize_maintenance_scheduling(max_residual_demand_per_week)
        maintenance_profiles.append(m.solution["maintenance"].to_series())
        status.append(m.termination_condition)
        key.append((bus, year))

UK00 2028


Writing binary variables.: 100%|██████████████████████████████| 1/1 [00:00<00:00, 80.32it/s]


Checking license ...


cpxchecklic: /lib64/libcurl.so.4: no version information available (required by cpxchecklic)



License found. [0.07 s]
Version identifier: 22.1.2.0 | 2024-12-10 | f4cec290b
CPXPARAM_Read_DataCheck                          1
CPXPARAM_LPMethod                                4
CPXPARAM_Threads                                 4
CPXPARAM_Emphasis_Numerical                      1
CPXPARAM_Emphasis_MIP                            4
CPXPARAM_MIP_Tolerances_MIPGap                   0.01
CPXPARAM_Barrier_ConvergeTol                     0.01
Tried aggregator 1 time.
MIQP Presolve eliminated 2 rows and 377 columns.
Reduced MIQP has 1116 rows, 137228 columns, and 307556 nonzeros.
Reduced MIQP has 136864 binaries, 0 generals, 0 SOSs, and 0 indicators.
Reduced MIQP objective Q matrix has 364 nonzeros.
Presolve time = 0.16 sec. (132.01 ticks)
Tried aggregator 1 time.
Reduced MIQP has 1116 rows, 137228 columns, and 307556 nonzeros.
Reduced MIQP has 136864 binaries, 0 generals, 0 SOSs, and 0 indicators.
Reduced MIQP objective Q matrix has 364 nonzeros.
Presolve time = 0.13 sec. (119.43 ticks)
Clas

...

Writing binary variables.: 100%|██████████████████████████████| 1/1 [00:00<00:00, 89.44it/s]


Checking license ...


cpxchecklic: /lib64/libcurl.so.4: no version information available (required by cpxchecklic)



License found. [0.08 s]
Version identifier: 22.1.2.0 | 2024-12-10 | f4cec290b
CPXPARAM_Read_DataCheck                          1
CPXPARAM_LPMethod                                4
CPXPARAM_Threads                                 4
CPXPARAM_Emphasis_Numerical                      1
CPXPARAM_Emphasis_MIP                            4
CPXPARAM_MIP_Tolerances_MIPGap                   0.01
CPXPARAM_Barrier_ConvergeTol                     0.01
Tried aggregator 1 time.
MIQP Presolve eliminated 4 rows and 699 columns.
Reduced MIQP has 1030 rows, 121576 columns, and 272425 nonzeros.
Reduced MIQP has 121212 binaries, 0 generals, 0 SOSs, and 0 indicators.
Reduced MIQP objective Q matrix has 364 nonzeros.
Presolve time = 0.15 sec. (134.94 ticks)
Tried aggregator 1 time.
Reduced MIQP has 1030 rows, 121576 columns, and 272425 nonzeros.
Reduced MIQP has 121212 binaries, 0 generals, 0 SOSs, and 0 indicators.
Reduced MIQP objective Q matrix has 364 nonzeros.
Presolve time = 0.12 sec. (105.93 ticks)
Clas

Dual values of MILP couldn't be parsed


In [342]:
country_plants, max_residual_demand_per_week, outage_days_country_plants, share_winter_country_plants = prepare_input_data('CH00', 2028)

In [356]:
country_plants, max_residual_demand_per_week, outage_days_country_plants, share_winter_country_plants = prepare_input_data('UK00', 2028)

In [357]:
country_plants_p_nom = country_plants.p_nom.div(1e3) 
max_residual_demand_per_week_normed = max_residual_demand_per_week.div(1e3)

In [358]:
m = linopy.Model()
plant = country_plants.index
    
maintenance = m.add_variables(binary=True, coords=[day, plant], name="maintenance")

In [359]:
daily_margin = m.add_variables(coords=[day], name="daily_margin")
    
days_winter = share_winter_country_plants.multiply(outage_days_country_plants).sort_values().round()

In [360]:
m.add_constraints(
    maintenance.loc[winter_time_days].sum("day")>=days_winter,
    name="winter_maintenance_days"
)

Constraint `winter_maintenance_days` [plant: 376]:
--------------------------------------------------
[87]: +1 maintenance[0, 87] + 1 maintenance[1, 87] + 1 maintenance[2, 87] ... +1 maintenance[362, 87] + 1 maintenance[363, 87] + 1 maintenance[364, 87]                                                                                                                               ≥ 2.0
[283]: +1 maintenance[0, 283] + 1 maintenance[1, 283] + 1 maintenance[2, 283] ... +1 maintenance[362, 283] + 1 maintenance[363, 283] + 1 maintenance[364, 283]                                                                                                                        ≥ 2.0
[284]: +1 maintenance[0, 284] + 1 maintenance[1, 284] + 1 maintenance[2, 284] ... +1 maintenance[362, 284] + 1 maintenance[363, 284] + 1 maintenance[364, 284]                                                                                                                        ≥ 2.0
[325]: +1 maintenance[0, 325] + 1 maintenance[

In [361]:
m.add_constraints(
    maintenance.sum("day") == outage_days_country_plants,
    name="maintenance_target"
)

Constraint `maintenance_target` [plant: 376]:
---------------------------------------------
[87]: +1 maintenance[0, 87] + 1 maintenance[1, 87] + 1 maintenance[2, 87] ... +1 maintenance[362, 87] + 1 maintenance[363, 87] + 1 maintenance[364, 87]                                                                                                                               = 54.0
[283]: +1 maintenance[0, 283] + 1 maintenance[1, 283] + 1 maintenance[2, 283] ... +1 maintenance[362, 283] + 1 maintenance[363, 283] + 1 maintenance[364, 283]                                                                                                                        = 27.0
[284]: +1 maintenance[0, 284] + 1 maintenance[1, 284] + 1 maintenance[2, 284] ... +1 maintenance[362, 284] + 1 maintenance[363, 284] + 1 maintenance[364, 284]                                                                                                                        = 27.0
[325]: +1 maintenance[0, 325] + 1 maintenance[1, 325]

In [362]:
m.add_constraints(
    maintenance.sum("plant").loc[364] == 0,
    name="no_maintenance_silvester"
)

Constraint `no_maintenance_silvester`
-------------------------------------
+1 maintenance[364, 87] + 1 maintenance[364, 283] + 1 maintenance[364, 284] ... +1 maintenance[364, UK00 oil exit 2036 2] + 1 maintenance[364, UK00 OCGT new 2028] + 1 maintenance[364, UK00 CCGT new 2028] = -0.0

In [363]:
m.add_constraints(
    #daily_margin == max_residual_demand_per_week - ((1 - maintenance)*country_plants.p_nom).sum("plant"),
    daily_margin == ((1 - maintenance)*country_plants_p_nom).sum("plant") - max_residual_demand_per_week_normed + constant,
    name="daily_margin"
)

Constraint `daily_margin` [day: 365]:
-------------------------------------
[0]: +1 daily_margin[0] + 2.676 maintenance[0, 87] + 0.68 maintenance[0, 283] ... +0.008 maintenance[0, UK00 oil exit 2036 2] + 1e-05 maintenance[0, UK00 OCGT new 2028] + 1e-05 maintenance[0, UK00 CCGT new 2028]               = 176.98592385860732
[1]: +1 daily_margin[1] + 2.676 maintenance[1, 87] + 0.68 maintenance[1, 283] ... +0.008 maintenance[1, UK00 oil exit 2036 2] + 1e-05 maintenance[1, UK00 OCGT new 2028] + 1e-05 maintenance[1, UK00 CCGT new 2028]               = 176.98592385860732
[2]: +1 daily_margin[2] + 2.676 maintenance[2, 87] + 0.68 maintenance[2, 283] ... +0.008 maintenance[2, UK00 oil exit 2036 2] + 1e-05 maintenance[2, UK00 OCGT new 2028] + 1e-05 maintenance[2, UK00 CCGT new 2028]               = 176.98592385860732
[3]: +1 daily_margin[3] + 2.676 maintenance[3, 87] + 0.68 maintenance[3, 283] ... +0.008 maintenance[3, UK00 oil exit 2036 2] + 1e-05 maintenance[3, UK00 OCGT new 2028] + 1e-05 mainte

In [364]:
m.add_objective(daily_margin**2)

In [351]:
m.solve(solver_name=solver_name, **solver_options)

Checking license ...


cpxchecklic: /lib64/libcurl.so.4: no version information available (required by cpxchecklic)



License found. [0.08 s]
Version identifier: 22.1.2.0 | 2024-12-10 | f4cec290b
CPXPARAM_Read_DataCheck                          1
CPXPARAM_LPMethod                                4
CPXPARAM_Threads                                 4
CPXPARAM_Emphasis_Numerical                      1
CPXPARAM_Emphasis_MIP                            4
CPXPARAM_MIP_Tolerances_MIPGap                   0.01
CPXPARAM_Barrier_ConvergeTol                     0.01
Tried aggregator 1 time.
MIQP Presolve eliminated 2 rows and 23 columns.
Reduced MIQP has 408 rows, 8372 columns, and 18338 nonzeros.
Reduced MIQP has 8008 binaries, 0 generals, 0 SOSs, and 0 indicators.
Reduced MIQP objective Q matrix has 364 nonzeros.
Presolve time = 0.01 sec. (8.32 ticks)
Tried aggregator 1 time.
Reduced MIQP has 408 rows, 8372 columns, and 18338 nonzeros.
Reduced MIQP has 8008 binaries, 0 generals, 0 SOSs, and 0 indicators.
Reduced MIQP objective Q matrix has 364 nonzeros.
Presolve time = 0.01 sec. (7.53 ticks)
Classifier predicts p

Dual values of MILP couldn't be parsed


('ok', 'optimal')

In [366]:
index = pd.MultiIndex.from_product([plants.bus.unique(), target_years])
maintenance_profiles = pd.concat(maintenance_profiles, keys=index)
maintenance_profiles = maintenance_profiles.reset_index(0, drop=True)

/tmp/ipykernel_1210938/500812968.py:2: FutureWarning: The behavior of pd.concat with len(keys) != len(objs) is deprecated. In a future version this will raise instead of truncating to the smaller of the two sequences
  maintenance_profiles = pd.concat(maintenance_profiles, keys=index)


In [378]:
for year in target_years:
        
    maintenance_profiles_year = maintenance_profiles.loc[year].unstack(1)
    maintenance_profiles_year.index = maintenance_profiles_year.index * 24
    maintenance_profiles_year = maintenance_profiles_year.reindex(range(8760)).ffill()

    maintenance_profiles_year = 1 - maintenance_profiles_year.astype(int)